In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words
from pythainlp.util import dict_trie

import numpy as np
import joblib
import re

In [2]:
test_class = "main_class"

words = ['ลดน้ำหนัก',
         'เบาหวาน',
         "คนแก่",
         "ผู้เฒ่า",
         "ไดเอท",
         "ความสูง",
         "ส่วนสูง",
         "โปรตีน",
         "ระบบภูมิคุ้มกัน",
         "กระเพาะอาหาร",
         "ออกกำลังกาย"
        ]
custom_words_list = set(thai_words())
custom_words_list.update(words)
trie = dict_trie(dict_source=custom_words_list)

ngram = 2
engine = "newmm"

In [3]:
import csv

models_accuracy = {}

# Open the CSV file in read mode
with open(f'{test_class}/{test_class}-model-accuracy.csv', mode='r') as file:
    # Create a CSV reader object
    reader = csv.reader(file)

    next(reader)

    # Iterate through the CSV rows and print them
    for row in reader:
        models_accuracy[row[0]] = float(row[1])

class_labels = []

# Open the CSV file in read mode
with open(f'{test_class}/{test_class}-class-labels.csv', mode='r') as file:
    # Create a CSV reader object
    reader = csv.reader(file)

    next(reader)

    # Iterate through the CSV rows and print them
    for row in reader:
        class_labels.append(row[0])

maxlen = 0

# Open the CSV file in read mode
with open(f'{test_class}/{test_class}-max-length.csv', mode='r') as file:
    # Create a CSV reader object
    reader = csv.reader(file)

    next(reader)

    # Iterate through the CSV rows and print them
    for row in reader:
        maxlen = int(row[0])

tokenizer = joblib.load(f'{test_class}/{test_class}-tokenizer.joblib')

In [4]:
def create_ngrams(tokens, n):
    ngrams = []
    for i in range(len(tokens) - n + 1):
        ngram = " ".join(tokens[i:i + n])
        ngrams.append(ngram)
    return ngrams

In [5]:
def create_ngram_interpolations(tokens, n):
    unigrams = tokens
    ngrams = create_ngrams(tokens, n)
    ngram_interpolations = []

    for i in range(len(unigrams)):
        for j in range(n):
            if i + j < len(unigrams):
                ngram_interpolations.append(" ".join(unigrams[i:i + j + 1]))

    return ngram_interpolations

In [6]:
print(maxlen)
print(models_accuracy)
print(class_labels)

20
{'FC': 0.9879879951477051, 'RNN': 0.9609609842300415, 'LSTM': 0.9939939975738525, 'bi-LSTM': 0.9939939975738525}
['0', '1', '2']


# Input Testing

In [7]:
input_text_list = ["ถ้ากินอาหารหวาน ๆ เยอะ ๆ จะทำให้อ้วนมากมั้ย",
                   "ในแต่ละวันควรรับประทานอาหารอะไร",
                   "อาหารแบบไหนทำให้อ้วน",
                   "ขอเมนูอาหารคลีนหน่อย",
                   "อาหารในแต่ละวันที่เรารับประทานควรมีสารอาหารอะไรบ้าง",
                   "เราควรกินข้าวหมูกรอบบ่อยแค่ไหน",
                   "สารอาหารหลักที่เรารับประทาน มาจากที่ไหนได้บ้าง",
                   "ผักและผลไม้สดมีสารอาหารต่างกันอย่างไร",
                   "ตอนเย็นกินข้าวมื้อหนัก ๆ ได้มั้ย",
                   "อาหารเช้าจำเป็นหรือไม่",
                   "อาหารมื้อเช้ากินอะไรดีจะได้อิ่ม ๆ"
                  ]

In [8]:
question = input()

 เป็นเบาหวานต้องออกกำลังกายยังไง


In [9]:
def preprocess(text):
    return re.sub(r'[\W_]+', '', text).lower()
    
def process_text_v2(input_text, this_dic):
    found_keys = []
    input_preprocessed = preprocess(input_text)
    for key, value in this_dic.items():
        for word in value:
            word_preprocessed = preprocess(word)
            if re.search(word_preprocessed, input_preprocessed):
                if key not in found_keys:
                    found_keys.append(key)
    return found_keys

def check_bmi(bmi_value , userId):
  bmi = int(bmi_value)
  print(userId , end = "==")
  if bmi < 18.5:
    print("น้ำหนักน้อยเกินไป (Underweight) < 18.5 : แสดงว่าร่างกายมีค่า BMI ต่ำกว่าเกณฑ์ หรือน้ำหนักน้อยต่ำกว่าเกณฑ์ หรือเป็นคนผอม")
  elif bmi <= 22.9:
    print("น้ำหนักปกติ (Normal weight) 18.5 - 22.9 : ถือว่ามีรูปร่างปกติสมส่วน เหมาะสำหรับผู้ชายไทย ซึ่งมีความเสี่ยงต่อโรคต่าง ๆ น้อยที่สุด")
  elif bmi <= 29.9:
    print("น้ำหนักเกิน (Overweight) 23 - 29.9 : แสดงว่าคน ๆ นั้นน้ำหนักเกิน กำลังก้าวสู่การเป็นคนอ้วน แม้จะยังไม่ใช่คนอ้วนแต่หากปล่อยไว้ ก็มีโอกาสเป็นคนอ้วนได้ และมีโอกาสจะเกิดโรคต่าง ๆ ได้ โดยเฉพาะหากคนในครอบครัวมีประวัติการเป็นโรคเบาหวาน ความดัน หรือโลหิตสูง เพราะมีความเสี่ยงมากกว่าคนปกติแล้ว")
  elif bmi <= 34.9:
    print("อ้วน (Obese) 30 - 34.9 : แสดงว่าร่างกายเข้าสู่ภาวะอ้วนระดับ 1 ถือว่าเข้าสู่ภาวะโรคอ้วน (Obesity) แล้ว ซึ่งถือเป็นความผิดปกติของร่างกาย ที่มีปริมาณไขมันสะสมตามอวัยวะส่วนต่าง ๆ เกินมาตรฐาน จำเป็นต้องมีการลดน้ำหนัก ควบคุมปริมาณน้ำตาล คาร์โบไฮเดรต และไขมัน เพื่อป้องกันการเกิดโรคเรื้อรัง และโรคแทรกซ้อนตามมา")
  else:
    print("อ้วนมาก (Morbidly obese) > 35 : แสดงว่าคน ๆ นั้น อยู่ในภาวะอ้วนระดับ 2 ถือว่าอ้วนมากขึ้นแล้ว เสี่ยงต่อการเกิดโรคร้ายแรงที่แฝงมากับความอ้วน จะต้องระวังการรับประทานไขมัน และควรออกกำลังกายสม่ำเสมอด้วย")

In [10]:
# for input_text in input_text_list:
#     print("Input Text:", input_text)
main_class = ""

criteria_value = {}

for each_model_name, accuracy in models_accuracy.items():
    model = load_model(f'{test_class}/{test_class}-{each_model_name}')

    tokens = word_tokenize(question, engine=engine, keep_whitespace=False, custom_dict=trie)  # Adjust the engine for Thai tokenization
    unigram = create_ngrams(tokens, 1)

    # Tokenize and pad
    input_text_tokenized = tokenizer.texts_to_sequences([unigram])
    input_text_padded = pad_sequences(input_text_tokenized, maxlen=maxlen, padding='post', truncating='pre', value=0)

    predictions = model.predict(input_text_padded)

    # Convert predictions to class labels
    predicted_class_index = np.argmax(predictions, axis=1)
    predicted_class_label = class_labels[predicted_class_index[0]]

    # Print predictions
    # print(f"{each_model_name}: {max(predictions[0])}")

    if max(predictions[0]) >= 0.7:
        # print(f"Predicted Class: {predicted_class_label}")
        criteria_value[max(predictions[0]) * accuracy] = predicted_class_label
#     else:
#         print("Predicted Class: Can't answer")

#     print("------------------------------------------")

print("Final Answer : ", end="")
if (len(criteria_value) == 0):
    print("Can't decide")
    main_class = "NOT FOUND"
else:
    # print(criteria_value[max(criteria_value.keys())])
    main_class = "class"+criteria_value[max(criteria_value.keys())]
print("main_class :" , main_class)

1/1 [==============================] - 0s 28ms/step


1/1 [==============================] - 0s 78ms/step


1/1 [==============================] - 0s 237ms/step


1/1 [==============================] - 0s 377ms/step
Final Answer : main_class : class2


In [11]:
import csv

models_accuracy = {}

# Open the CSV file in read mode
with open(f'{main_class}/{main_class}-model-accuracy.csv', mode='r') as file:
    # Create a CSV reader object
    reader = csv.reader(file)

    next(reader)

    # Iterate through the CSV rows and print them
    for row in reader:
        models_accuracy[row[0]] = float(row[1])

class_labels = []

# Open the CSV file in read mode
with open(f'{main_class}/{main_class}-class-labels.csv', mode='r') as file:
    # Create a CSV reader object
    reader = csv.reader(file)

    next(reader)

    # Iterate through the CSV rows and print them
    for row in reader:
        class_labels.append(row[0])

maxlen = 0

# Open the CSV file in read mode
with open(f'{main_class}/{main_class}-max-length.csv', mode='r') as file:
    # Create a CSV reader object
    reader = csv.reader(file)

    next(reader)

    # Iterate through the CSV rows and print them
    for row in reader:
        maxlen = int(row[0])

tokenizer = joblib.load(f'{main_class}/{main_class}-tokenizer.joblib')

In [12]:
# for input_text in input_text_list:
#     print("Input Text:", input_text)
test_class = main_class

if (main_class != "NOT FOUND"):
    subclass = ""

    criteria_value = {}

    for each_model_name, accuracy in models_accuracy.items():
        model = load_model(f'{test_class}/{test_class}-{each_model_name}')

        tokens = word_tokenize(question, engine=engine, keep_whitespace=False, custom_dict=trie)  # Adjust the engine for Thai tokenization
        unigram = create_ngrams(tokens, 1)

        # Tokenize and pad
        input_text_tokenized = tokenizer.texts_to_sequences([unigram])
        input_text_padded = pad_sequences(input_text_tokenized, maxlen=maxlen, padding='post', truncating='pre', value=0)

        predictions = model.predict(input_text_padded)

        # Convert predictions to class labels
        predicted_class_index = np.argmax(predictions, axis=1)
        predicted_class_label = class_labels[predicted_class_index[0]]

        # Print predictions
        # print(f"{each_model_name}: {max(predictions[0])}")

        if max(predictions[0]) >= 0.7:
            # print(f"Predicted Class: {predicted_class_label}")
            criteria_value[max(predictions[0]) * accuracy] = predicted_class_label
    #     else:
    #         print("Predicted Class: Can't answer")

    #     print("------------------------------------------")

    print("Final Answer : ", end="")
    if (len(criteria_value) == 0):
        print("Can't decide")
        subclass = "NOT FOUND"
    else:
        # print(criteria_value[max(criteria_value.keys())])
        subclass = "class"+criteria_value[max(criteria_value.keys())]
    print("subclass :", subclass)

1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 72ms/step


1/1 [==============================] - 0s 243ms/step


1/1 [==============================] - 0s 375ms/step
Final Answer : subclass : class2005


In [13]:
dic_1001 = {"1": ["โอเมก้า 3" , "โอเมก้าสาม" , "OMEGA 3" , "ดีเอสเอ" , "DSA" , "อีพีเอส", "EPS" , "กรดไขมันดีเอสเอ" , "กรดไขมันอีพีเอส" , "กรดไขมันแอลฟาไลโนเลนิก" , "แอลฟาไลโนเลนิก" , "กรดไขมัน"],
            "2": ["โอเมก้า 6" , "โอเมก้าหก" , "OMEGA 6" , "กรดไขมัน" , "ไลโนอิกแอซิด" , "แกมมา ไลโนเลอิก แอซิด" , "LINOLEIC ACID" , "GAMMA LINOLEIC ACID"],
            "3": ["ไนตริกออกไซด์" , "ไนตริกออกไซต์" , "NITRIC OXIDE", "ไนตริก"],
            "4": ["วิตามิน" , "VITAMIN", "แร่ธาตุ", "MINERAL", "เกลือแร่" , "MINERAL SALT"],
            "5": ["สารอาหาร" , "NUTRITION"] ,
            "6" : ["ไขมันดี" ,  "HDL" , "คอเลสเตอรอลดี" ,"เฮชดีแอล"] ,
            "7" :[ "คาร์โบไฮเดรต" , "CARBOHYDRATE"] ,
            "8" : ["แคลเซียม" , "CALCIUM"] ,
            "9" : ["โปรตีน" , "PROTIEN"] ,
            "10" : ["โฟแลต" , "วิตามินบี 9" , "วิตามินB9" , "วิตามินบีเก้า"] ,
            "11" : ["ใยอาหาร", "กากใย"] ,
            "12" : ["ต้านอนุมูลอิสระ", "อนุมูลอิสระ"] ,
            "13" : ["ธาตุเหล็ก", "เหล็ก"]}

dic_2003 = {"1" : ["แขน"] ,
            "2" : ["กล้ามอก" , "หน้าอก" , "เนื้ออก" , "กล้ามเนื้ออก" , "ส่วนบนลำตัว" , "ช่วงบนลำตัว"] ,
            "3" : ["ขาใหญ่" , "ต้นขา" , "ขาเผละ" , "เชิงกราน" , "ขาเรียว" , "ขาเบียด" , "น่องขา" , "เนื้อขา" , "สะโพก"] ,
            "4" : ["หลัง" , "ส่วนกลางลำตัว" , "ช่วงกลางลำตัว"] ,
            "5" : ["ท้อง"  , "SIX PACK" , "พุง" , "ร่อง11" ,"เอว" , "ซิกแพค", "ส่วนล่างของลำตัว"]}

dic_2006 = {"1" : ["เข่า"] ,
            "2" : ["ไหล่" , "บ่า" , "คอ" , "ออฟฟิศซินโดรม" , "OFFICE SYNDROME" , "สะบัก"] ,
            "3" : ["หลัง"]}

In [14]:
from answer_class_1001 import dic_class_1001
from answer_class_2003 import dic_class_2003
from answer_class_2006 import dic_class_2006
from answer_class_1 import dic_class_1
from answer_class_2 import dic_class_2
from answer_class_0 import dic_class_0

result = ""
if question.startswith("!@#$%"):
    userId = question[5:38] # there are 32 character for each user id
    question = question[38:]
    weight , height = question.split()
    bmi = weight/((height/100)**2)
    check_bmi(bmi)
# check subclass before
else:
    if subclass.startswith(main_class) and subclass != "NOT FOUND" and main_class != "NOT FOUND" and main_class != "" and subclass != "":
        if subclass == "class1001":
            result = process_text_v2(question , dic_1001)
            for final_class in result:
              print(dic_class_1001[final_class])

        elif subclass == "class2003":
            result = process_text_v2(question , dic_2003)
            for final_class in result:
              print(dic_class_2003[final_class])
        elif subclass == "class2006":
            result = process_text_v2(question , dic_2006)
            
            for final_class in result:
              print(dic_class_2006[final_class])
        elif subclass.startswith("class0"): #001 , 002 check ด้วยว่า subclass มันตรงกับ main_class มั้ย
            # check final and main class
            if (subclass == "class002"):
              print("!@#$%โปรดระบุน้ำหนัก ส่วนสูง ของคุณตามในหน่วยกิโลกรัมและเซนติเมตรลำดับ เช่น 56 175")
            # print("read file in class 0")
            else:
                subclass = subclass[5:]
                print(dic_class_0[subclass])
        elif subclass.startswith("class1"):
            subclass = subclass[5:]
            # print("read file in class 1")
            print(dic_class_1[subclass])
        elif subclass.startswith("class2"):
            subclass = subclass[5:]
            # print("read file in class 2")
            print(dic_class_2[subclass])
        else:
            result = "NOT FOUND"
        # print(result)
    else:
        print("FINAL ANSWER : NOT FOUND")

ท่าบริหารร่างกายที่เหมาะกับผู้ป่วยเบาหวาน ได้แก่ 
- ท่ายืดกล้ามเนื้อ เช่น ท่ายืดหลัง ท่ายืดขา ท่ายืดไหล่ เพื่อเพิ่มการเคลื่อนไหวของร่างกายและลดความเสี่ยงในการบาดเจ็บ
- ท่าบริหารกล้ามเนื้อแกนกลางลำตัว เช่น แพลงก์ สควอต ซิทอัพ เพื่อเสริมสร้างกล้ามเนื้อแกนกลางลำตัวและควบคุมการเคลื่อนไหวของร่างกาย
- ท่าบริหารกล้ามเนื้อขา เช่น เดิน วิ่ง ปั่นจักรยาน เพื่อเสริมกล้ามเนื้อขาและควบคุมระดับน้ำตาลในเลือด.
- ท่าบริหารกล้ามเนื้อแขน เช่น วิดพื้น ดึงข้อ โยคะ เพื่อเสริมกล้ามเนื้อแขนและควบคุมระดับน้ำตาลในเลือด.

โดยการออกกำลังกายอย่างสม่ำเสมอเป็นสิ่งสำคัญสำหรับผู้ป่วยเบาหวาน เพื่อควบคุมระดับน้ำตาลในเลือด ลดน้ำหนัก และลดความเสี่ยงในการเกิดโรคแทรกซ้อนอื่น ๆ ของโรคเบาหวานได้อย่างดี
